In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env

/Users/tarikh/Code/whatspie/whatspie-support-ai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
api_key = os.getenv("GOOGLE_API_KEY")
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=api_key)

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import create_retrieval_chain

# Text loader
# load the document and split it into chunks
loader = TextLoader("./whatspie.md")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)


Created a chunk of size 794, which is longer than the specified 500
Created a chunk of size 535, which is longer than the specified 500
Created a chunk of size 551, which is longer than the specified 500
Created a chunk of size 734, which is longer than the specified 500
Created a chunk of size 1084, which is longer than the specified 500
Created a chunk of size 3995, which is longer than the specified 500
Created a chunk of size 1319, which is longer than the specified 500
/Users/tarikh/Code/whatspie/whatspie-support-ai/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnablePassthrough

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
message = """
Kamu adalah Customer Support AI dari whatspie yang siap menjawab pertanyaan apapun, jika pertanyaan nya kurang jelas, tolong tanyakan lagi dan jangan menjawab sesuatu yang kurang pasti.\
Kamu hanya bisa menjawab tentang context yang ada. kemudian jawaban yang kamu berikan harus dalam bahasa indonesia

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "Kamu adalah Customer Support AI dari whatspie yang siap menjawab pertanyaan apapun, jika pertanyaan nya kurang jelas, tolong tanyakan lagi dan jangan menjawab sesuatu yang kurang pasti.",
#         ),
#         MessagesPlaceholder(variable_name="messages"),
#     ]
# )
retriever = db.as_retriever()

# chain =   prompt | model | retrieval_chain
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

response = rag_chain.invoke("tell me about cats")

print(response.content)

# chain.invoke({"messages":["Hello"]})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.ge

KeyboardInterrupt: 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


In [ ]:
response = rag_chain.invoke("Nama saya iik", config=config)

print(response.content)

Halo Iik, apa yang ingin kamu tanyakan tentang Whatspie?  

Silahkan berikan pertanyaan yang lebih spesifik, agar saya bisa membantu kamu dengan lebih baik. 



In [ ]:
response = rag_chain.invoke("Siapa nama saya?", config=config)

print(response.content)

Maaf, tapi saya tidak bisa menemukan informasi tentang nama Anda dalam konteks yang Anda berikan. Bisakah Anda memberikan informasi lebih lanjut? 



In [ ]:
with_message_history = RunnableWithMessageHistory(rag_chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Kamu")],
    config=config,
)

response.content


AttributeError: 'list' object has no attribute 'replace'